In [18]:
import pickle as pkl
import mujoco
from mujoco import mjx
import jax
from jax import numpy as jnp
import dm_control
from dm_control import mjcf
from dm_control.locomotion.walkers import rescale
import numpy as np
import importlib
utils = importlib.import_module("stac-mjx.utils")
from jax.tree_util import Partial
from functools import partial
from  mujoco.mjx._src import smooth


In [19]:
utils.init_params("params/params.yaml")
mjmodel = mujoco.MjModel.from_xml_path("././models/rodent_stac.xml")
mjmodel.opt.solver = mujoco.mjtSolver.mjSOL_NEWTON
mjmodel.opt.iterations = 1
mjmodel.opt.ls_iterations = 4
mjdata = mujoco.MjData(mjmodel)
mjx_model = mjx.put_model(mjmodel)
mjx_data = mjx.make_data(mjx_model)
mjx_data.qpos

Array([3.0995261e-02, 2.0585249e-07, 6.8089575e-02, 1.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [20]:
# open and set params
with open("././pose_opt_qs.p", "rb") as file:
    data = pkl.load(file)

# mjx_model = data["mjx_model"]
# mjx_data = data["mjx_data"]
# kp_data = data["kp_data"]
# offsets = data["offsets"]
# q = data["qpos"]
# x = data["xpos"]
# physics = data["physics"]
# walker_body_sites = data["walker_body_sites"]

# utils.params["site_index_map"] = data["site_index_map"]
# utils.params["part_names"] = data["names_qpos"]

AttributeError: 'tuple' object has no attribute 'model'

In [ ]:
mjx_data = mjx.euler(mjx_model, mjx_data)
mjx_data.qpos


In [ ]:
mjcf_root = mjcf.from_path(params["XML_PATH"])

# Rescale
rescale.rescale_subtree(
        mjcf_root,
        params["SCALE_FACTOR"],
        params["SCALE_FACTOR"],
    )

In [ ]:
# This function modifies mjcf in place so root should be passed in to make sure we're doing it right
def set_body_sites(root, params):
    # gets part names (but only )
    # part_names = physics.named.data.qpos.axes.row.names
    # for _ in range(6):
    #     part_names.insert(0, part_names[0])
    # print(part_names)
    # mjcf_root = mjcf.from_path(params["XML_PATH"])
    body_sites = []
    for key, v in params["KEYPOINT_MODEL_PAIRS"].items():
        parent = root.find("body", v)
        
        pos = params["KEYPOINT_INITIAL_OFFSETS"][key]
        
        site = parent.add(
            "site",
            name=key,
            type="sphere",
            size=[0.005],
            rgba="0 0 0 1",
            pos=pos,
            group=3,
        )
        body_sites.append(site)
    return body_sites

In [ ]:
root = mjcf_root.__copy__()
body_sites = set_body_sites(root, params)

In [ ]:
physics = mjcf.Physics.from_mjcf_model(root)
binding = physics.bind(body_sites)

In [ ]:
binding.pos

SynchronizingArrayWrapper([[-0.03230154, -0.00472705, -0.02205959],
                           [-0.03230154,  0.00472705, -0.02205959],
                           [ 0.        ,  0.        ,  0.        ],
                           [ 0.        ,  0.        ,  0.        ],
                           [-0.01593121,  0.01035529, -0.02230369],
                           [-0.01593121, -0.01035529, -0.02230369],
                           [ 0.02109994,  0.00761433, -0.00275217],
                           [ 0.02109994, -0.00761433, -0.00275217],
                           [ 0.00303175,  0.00151587, -0.0083373 ],
                           [ 0.00303175, -0.00151587, -0.0083373 ],
                           [-0.015     ,  0.015     ,  0.        ],
                           [-0.015     , -0.015     ,  0.        ],
                           [ 0.01542265,  0.017479  , -0.02570441],
                           [ 0.01542265, -0.017479  , -0.02570441],
                           [ 0.0287    ,  0.0098

In [ ]:
params = utils.load_params("params/params.yaml")
mjmodel = mujoco.MjModel.from_xml_path(params["XML_PATH"])
mjdata = mujoco.MjData(mjmodel)
mjdata.site_xpos.shape

(86, 3)

In [ ]:
physics.named.data.xpos

FieldIndexer(xpos):
                         x         y         z         
 0               world [ 0         0         0       ]
 1               torso [ 0.0279    1.85e-07  0.0586  ]
 2          vertebra_1 [ 0.0106    1.6e-08   0.0737  ]
 3          vertebra_2 [ 0.00557   2.35e-06  0.0742  ]
 4          vertebra_3 [-0.000298 -7.02e-06  0.0741  ]
 5          vertebra_4 [-0.0064    8.53e-07  0.073   ]
 6          vertebra_5 [-0.0126   -4.73e-06  0.0707  ]
 7          vertebra_6 [-0.0182   -1.62e-09  0.0674  ]
 8              pelvis [-0.0299    3.16e-06  0.0601  ]
 9         upper_leg_L [-0.0343    0.0103    0.0512  ]
10         lower_leg_L [-0.0204    0.0261    0.0281  ]
11              foot_L [-0.0495    0.0218    0.00825 ]
12               toe_L [-0.0305    0.0287    0.00577 ]
13         upper_leg_R [-0.0343   -0.0103    0.0512  ]
14         lower_leg_R [-0.0204   -0.0261    0.0281  ]
15              foot_R [-0.0495   -0.0218    0.00825 ]
16               toe_R [-0.0305   -0.0287   

In [ ]:
# I <3 dict comprehension
axis = physics.named.model.site_pos._axes[0]
site_index_map = {key: axis.convert_key_item(key) for key in params["KEYPOINT_MODEL_PAIRS"].keys()}
site_index_map

{'AnkleL': 20,
 'AnkleR': 31,
 'EarL': 77,
 'EarR': 78,
 'ElbowL': 86,
 'ElbowR': 100,
 'FootL': 23,
 'FootR': 34,
 'HandL': 92,
 'HandR': 106,
 'HipL': 11,
 'HipR': 12,
 'KneeL': 17,
 'KneeR': 28,
 'ShoulderL': 83,
 'ShoulderR': 97,
 'Snout': 79,
 'SpineF': 68,
 'SpineL': 13,
 'SpineM': 2,
 'TailBase': 14,
 'WristL': 89,
 'WristR': 103}

In [ ]:
physics.named.model.site_pos[14]

array([0., 0., 0.])

In [ ]:
offsets = np.copy([physics.model.site_pos[i] for i in site_index_map.values()])
# offsets *= params['SCALE_FACTOR']
offsets

array([[-0.03230154, -0.00472705, -0.02205959],
       [-0.03230154,  0.00472705, -0.02205959],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-0.01593121,  0.01035529, -0.02230369],
       [-0.01593121, -0.01035529, -0.02230369],
       [ 0.02109994,  0.00761433, -0.00275217],
       [ 0.02109994, -0.00761433, -0.00275217],
       [ 0.00303175,  0.00151587, -0.0083373 ],
       [ 0.00303175, -0.00151587, -0.0083373 ],
       [-0.015     ,  0.015     ,  0.        ],
       [-0.015     , -0.015     ,  0.        ],
       [ 0.01542265,  0.017479  , -0.02570441],
       [ 0.01542265, -0.017479  , -0.02570441],
       [ 0.0287    ,  0.00984   , -0.02542   ],
       [ 0.0287    , -0.00984   , -0.02542   ],
       [ 0.        ,  0.        ,  0.        ],
       [-0.015     ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.   

In [ ]:
binding.pos

SynchronizingArrayWrapper([[-0.03230154, -0.00472705, -0.02205959],
                           [-0.03230154,  0.00472705, -0.02205959],
                           [ 0.        ,  0.        ,  0.        ],
                           [ 0.        ,  0.        ,  0.        ],
                           [-0.01593121,  0.01035529, -0.02230369],
                           [-0.01593121, -0.01035529, -0.02230369],
                           [ 0.02109994,  0.00761433, -0.00275217],
                           [ 0.02109994, -0.00761433, -0.00275217],
                           [ 0.00303175,  0.00151587, -0.0083373 ],
                           [ 0.00303175, -0.00151587, -0.0083373 ],
                           [-0.015     ,  0.015     ,  0.        ],
                           [-0.015     , -0.015     ,  0.        ],
                           [ 0.01542265,  0.017479  , -0.02570441],
                           [ 0.01542265, -0.017479  , -0.02570441],
                           [ 0.0287    ,  0.0098

In [ ]:
@jax.vmap
def blah(vec):
    return mjx_model

jit_blah = jax.jit(blah)

In [ ]:
mjxm = jit_blah(jnp.zeros(10))

In [ ]:
mjxm.__dict__.keys()

dict_keys(['nq', 'nv', 'nu', 'na', 'nbody', 'njnt', 'ngeom', 'nmesh', 'npair', 'nexclude', 'neq', 'nnumeric', 'nM', 'opt', 'stat', 'qpos0', 'qpos_spring', 'body_parentid', 'body_rootid', 'body_weldid', 'body_jntnum', 'body_jntadr', 'body_dofnum', 'body_dofadr', 'body_geomnum', 'body_geomadr', 'body_pos', 'body_quat', 'body_ipos', 'body_iquat', 'body_mass', 'body_subtreemass', 'body_inertia', 'body_invweight0', 'jnt_type', 'jnt_qposadr', 'jnt_dofadr', 'jnt_bodyid', 'jnt_limited', 'jnt_actfrclimited', 'jnt_solref', 'jnt_solimp', 'jnt_pos', 'jnt_axis', 'jnt_stiffness', 'jnt_range', 'jnt_actfrcrange', 'jnt_margin', 'dof_bodyid', 'dof_jntid', 'dof_parentid', 'dof_Madr', 'dof_solref', 'dof_solimp', 'dof_frictionloss', 'dof_armature', 'dof_damping', 'dof_invweight0', 'dof_M0', 'geom_type', 'geom_contype', 'geom_conaffinity', 'geom_condim', 'geom_bodyid', 'geom_priority', 'geom_solmix', 'geom_solref', 'geom_solimp', 'geom_size', 'geom_pos', 'geom_quat', 'geom_friction', 'geom_margin', 'geom_ga

In [ ]:
d = {"one": "1", "two": "2", "three": "2"}

In [ ]:
def d_f(x, d):
    return x, d["one"]

In [ ]:
import functools
x = jnp.array([1,3,5,7,9])
# partial_f = functools.partial(d_f, d=d)

In [ ]:
# jax.vmap(partial_f)(x)

(Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 Array([1, 1, 1, ..., 1, 1, 1], dtype=int32, weak_type=True))

In [ ]:
def innerf(d):
    return int(d["one"])

In [ ]:
def outerf(y, d):
    return y, innerf(d)

In [ ]:
def outererf(x, d):
    # p_outerf = functools.partial(outerf, d=d)
    return outerf(x, d)

In [ ]:
partialer_f = functools.partial(outererf, d=d)
vmap_partialer_f = jax.vmap(partialer_f)

In [ ]:
jit_vmap_partialer_f = jax.jit(vmap_partialer_f)

In [ ]:
jit_vmap_partialer_f(x)

(Array([1, 3, 5, 7, 9], dtype=int32),
 Array([1, 1, 1, 1, 1], dtype=int32, weak_type=True))

In [ ]:
jax.jit(outerf)(x, d)

TypeError: Argument '1' of type <class 'str'> is not a valid JAX type

### TLDR:

You can safely pass in a dictionary to an inner function, as long as the dict is not an arg in the outer function. So, use functools.partial to pass in the dict to the outer function, then vmap after:

`func = jax.vmap(functools.partial(f, d=d))`

In [ ]:
jit_forward = jax.jit(lambda x, y: mjx.forward(x, y))

In [ ]:
jit_forward(mjx_model, mjx_data)

Data(solver_niter=Array(0, dtype=int32), ne=0, nf=0, nl=67, nefc=19427, ncon=4840, time=Array(0., dtype=float32), qpos=Array([3.0995261e-02, 2.0585249e-07, 6.5089576e-02, 1.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0

In [ ]:
%timeit jit_forward(mjx_model, mjx_data)

4.79 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit mjx.forward(mjx_model, mjx_data)

KeyboardInterrupt: 